In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import os
import pickle
import json
import re
import gc
from PIL import Image

In [2]:
tf.test.is_gpu_available()

True

In [3]:
# create giant dictionary for all data
data_dir = '../facebook_challenge_data/'
model_dir = 'models/'

# load data and print sizes
def get_dict(path):
    jsonl_content = open(path, 'r').read()
    data = [json.loads(jline) for jline in jsonl_content.split('\n')]
    return {datum['id'] : datum for datum in data}


train_dict = get_dict(data_dir + 'train.jsonl')
val_dict = get_dict(data_dir + 'dev.jsonl')
test_dict = get_dict(data_dir + 'test.jsonl')

print(len(train_dict))
print(len(val_dict))
print(len(test_dict))

8500
500
1000


In [4]:
# load text_roberta
import transformers
from tensorflow.keras import layers

MAX_SEQ_LEN = 50

class ROBERTA(transformers.TFRobertaModel):

    def __init__(self, config, *inputs, **kwargs):
        super(ROBERTA, self).__init__(config, *inputs, **kwargs)
        self.roberta.call = tf.function(self.roberta.call)

input_ids = layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32)
roberta = ROBERTA.from_pretrained('roberta-base')
roberta_encodings = roberta([input_ids])[0]
doc_encoding = tf.squeeze(roberta_encodings[:, 0:1, :], axis=1) # Keep [CLS] token encoding
doc_encoding = layers.Dropout(0.1)(doc_encoding) # Apply dropout
outputs = layers.Dense(1, activation='sigmoid', name='outputs')(doc_encoding)
text_model = tf.keras.models.Model(inputs=[input_ids], outputs=[outputs])
text_model.load_weights(model_dir + 'best_text_roberta_weights.h5')

print(text_model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
roberta (ROBERTA)            ((None, 50, 768), (None,  124645632 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1, 768)]          0         
_________________________________________________________________
tf_op_layer_Squeeze (TensorF [(None, 768)]             0         
_________________________________________________________________
dropout_38 (Dropout)         (None, 768)               0         
_________________________________________________________________
outputs (Dense)              (None, 1)                 769       
Total params: 124,646,401
Trainable params: 124,646,401
Non-trainable params: 0
_______________________________________________

In [5]:
# load cnn
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import layers
from tensorflow.keras.models import Model

conv_base = tf.keras.applications.resnet.ResNet152(include_top=False, 
                                                        weights='imagenet', 
                                                        input_shape=(224, 224, 3))

for layer in conv_base.layers:
    if isinstance(layer, layers.BatchNormalization): layer.trainable = True
    else: layer.trainable = False


img_model = Sequential()
img_model.add(conv_base)
img_model.add(layers.GlobalAveragePooling2D())
img_model.add(Dense(1024, activation='relu'))
img_model.add(Dense(512, activation='relu'))
img_model.add(Dense(1, activation='sigmoid'))
print(img_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152 (Model)            (None, 7, 7, 2048)        58370944  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total params: 60,994,433
Trainable params: 2,623,489
Non-trainable params: 58,370,944
_________________________________________________________________
None


In [6]:
# create new models that are all but the last layer
from tensorflow.keras.models import Sequential, Model

img_net = Sequential()
for layer in img_model.layers[:-1]: img_net.add(layer)
print(img_net.summary())

# free up memory from old models:
del img_model
gc.collect()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152 (Model)            (None, 7, 7, 2048)        58370944  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
Total params: 60,993,920
Trainable params: 2,622,976
Non-trainable params: 58,370,944
_________________________________________________________________
None


79

In [7]:
img_input = layers.Input((img_net.layers[0].input_shape[1:]))
img_embed = img_net(img_input)
x = layers.concatenate([doc_encoding, img_embed])
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)

prediction = layers.Dense(1, activation='sigmoid')(x)
fcm_model = Model(inputs=[input_ids, img_input], outputs=prediction)
print(fcm_model.summary())

optimizer = Adam(lr = 1e-6)
fcm_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
roberta (ROBERTA)               ((None, 50, 768), (N 124645632   input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 1, 768)]     0           roberta[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_Squeeze (TensorFlow [(None, 768)]        0           tf_op_layer_strided_slice[0][0]  
____________________________________________________________________________________________

In [8]:
from random import randint # for random cropping
from tensorflow.keras.preprocessing.sequence import pad_sequences

class FBMMDataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data_dict, tokenizer, pad_len, batch_size=32, dim=(299, 299), n_channels=3, shuffle=True):
        'Initialization'
        self.dim = dim
        self.data_dict = data_dict
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.pad_len = pad_len
        self.tokenizer = tokenizer
        
        # build labels list and id list
        self.id_list = list(self.data_dict.keys())
        self.labels = {ID: self.data_dict[ID]['label'] for ID in self.id_list}
        self.img_list = {ID: self.data_dict[ID]['img'] for ID in self.id_list}
            
        # get text dictionary
        self.text_dict = self.process_text(self.id_list)
        
        self.on_epoch_end()
        self.classes = [self.labels[self.id_list[i]] for i in self.indexes]

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.id_list) / self.batch_size)) + 1 # last batch is partial

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:index*self.batch_size + self.batch_size]
        
        
        # Find list of IDs
        id_list_temp = [self.id_list[k] for k in indexes]

        # Generate data
        X_txt, X_img, y = self.__data_generation(id_list_temp)
        
        return (X_txt, X_img), y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.id_list))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, id_list_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X_img = np.empty((len(id_list_temp), *self.dim, self.n_channels))
        X_txt = np.empty((len(id_list_temp), self.pad_len))
        y = np.empty(len(id_list_temp), dtype=int)

        # Generate data
        for i, ID in enumerate(id_list_temp):
            # Store sample
            X_img[i,] = self.process_img(data_dir + self.img_list[ID])
            X_txt[i,] = self.text_dict[ID]

            # Store class
            y[i] = self.labels[ID]

        return X_txt.astype(int), X_img, y
    
    def process_img(self, path): # method for getting image
        img = Image.open(path)
        img.load()
        img = img.resize(self.dim, Image.ANTIALIAS)
        data = np.asarray(img, dtype='uint8')
        im = self.augment(data)
        
        
        if im.shape==(self.dim[0], self.dim[1]): im = np.stack((im,)*3, axis=-1) # handle grayscale
        if im.shape == (*self.dim, 4): im = im[:,:,:3] # handle weird case
        
        return im
    
    def augment(self, im): # random crop and random mirror
        
        # random crop
        x_max, y_max = im.shape[0], im.shape[1]
        x_start, y_start = randint(0, x_max - self.dim[0]), randint(0, y_max - self.dim[1])
        im = im[x_start:x_start + self.dim[0], y_start:y_start + self.dim[1]]
        
        # random mirror
        if randint(0,1): im = np.flip(im, axis=1)
        
        return im
    
    def process_text(self, id_list):
        
        # matrix for texts
        texts = [self.data_dict[ID]['text'] for ID in id_list]
        sequences = [self.tokenizer.encode(text) for text in texts] # make this more efficient...
        text_seqs = pad_sequences(sequences, maxlen=self.pad_len)
        
        id_to_seq = {ID: txt for (ID, txt) in zip(id_list, text_seqs)} # map id to text seq
        
        return id_to_seq

In [9]:
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, TFRobertaModel
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
max_len = 50

# create data generators
train_gen = FBMMDataGenerator(data_dict=train_dict,
                          tokenizer=tokenizer,
                          pad_len=max_len,
                          batch_size=16,
                          dim=(224, 224),
                          n_channels=3,
                          shuffle=True)

val_gen = FBMMDataGenerator(data_dict=val_dict,
                          tokenizer=tokenizer,
                          pad_len=max_len,
                          batch_size=16,
                          dim=(224, 224),
                          n_channels=3,
                          shuffle=True)

In [ ]:
# train model
from tensorflow.keras.callbacks import ModelCheckpoint

mcp_save = ModelCheckpoint(model_dir + 'best_roberta_fusion.h5', 
                           save_weights_only=True, 
                           save_best_only=True, 
                           verbose=1,
                           monitor='val_loss', 
                           mode='min')

history = fcm_model.fit_generator(train_gen,
                    validation_data=val_gen,
                    shuffle=True,
                    epochs=5,
                    callbacks=[mcp_save])

Epoch 1/5
219/532 [===========>..................] - ETA: 9:06 - loss: 0.6286 - accuracy: 0.6461

In [ ]:
plt.plot(history.history['accuracy'], label = 'acc')
plt.plot(history.history['val_accuracy'], label = 'val acc')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.legend()
plt.show()

In [ ]:
# fcm_model.load_weights(model_dir + 'best_roberta_fusion.h5')

In [ ]:
# test
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import math

y_val = val_gen.classes

# get AUROC
preds = fcm_model.predict_generator(val_gen)
print('Val AUROC:', roc_auc_score(y_val, preds))

# get loss and acc
preds_bin = np.array(preds)
preds_bin[preds>0.5] = 1
preds_bin[preds<=0.5] = 0
print('Val Accuracy:', accuracy_score(y_val, preds_bin))